<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/Data_Ac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing python libraries that will be use to get data from yahoo finance

In [1]:
! pip install yahooquery
! pip install urllib3==1.26
! pip install requests==2.25
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Importing modules that will be used to filter data

In [2]:
from yahooquery import Ticker
from bs4 import BeautifulSoup

import concurrent.futures
import pandas as pd
import requests
import re

Scrapping Data from Infomoney website, getting a list of stocks from IBOVESPA.

In [3]:
url = requests.get('https://www.infomoney.com.br/cotacoes/empresas-b3/')

soup = BeautifulSoup(url.text, 'html.parser')

ticker_name_list = soup.find_all('td', class_ = 'strong')

ticker = str(ticker_name_list)

tickers = re.findall('>.*</',ticker)

tkrs = []

# strnot = ("11","F","34")
for i in tickers:
  if "11" not in i and "F" not in i:
    tkr = i.strip("</>")
    tkr = tkr + ".SA"
    tkrs.append(tkr)

In [34]:
def fetch_historical_financial(tickers):
    # Yahoo finance requisition getting companies ticker
    # sumary = Ticker(tickers)
    sumary = Ticker('aapl')

    # getting companies financial data
    data = sumary.financial_data 

    df = pd.DataFrame(data,  index=[0])

    return df

In [5]:
def fetch_historical_balance(tickers):
    # Balance Ticker lists
    balance = Ticker(tickers)

    # Getting companies balance data
    data = balance.balance_sheet(frequency='a',trailing=False)

    df = pd.DataFrame(data)

    return df

In [32]:
def fetch_historical_finandata(tickers):
    # Balance Ticker lists
    financial = Ticker(tickers)

    # Getting companies balance data
    types = ['TotalDebt', 'TotalAssets', 'EBIT', 'EBITDA', 'PeRatio']
    data = financial.get_financial_data(types,trailing=False)

    df = pd.DataFrame(data)

    return df.T

In [7]:
def fetch_historical_price(tickers):
    pricetkrs = Ticker(tickers)

    dataprice = pricetkrs.price

    df = pd.DataFrame(dataprice)

    return df.T

In [8]:
# Define a function to fetch data for each batch of tickers
def fetch_data_batch(tkrs,FS):
    if FS == 1:
        data = fetch_historical_financial(tkrs)

    elif FS == 2:
        data = fetch_historical_balance(tkrs)

    elif FS == 3:
        data = fetch_historical_finandata(tkrs)

    elif FS == 4:
        data = fetch_historical_price(tkrs)

    return data

In [9]:
def ElementsList(elements_per_list,tkrs):
    # Divide the ticker list into smaller lists
    smaller_lists = [tkrs[i:i+elements_per_list] for i in range(0, len(tkrs), elements_per_list)]
    
    return smaller_lists

In [23]:
# # # Function that processes operations based on different flags
# # # BS: Batch size
# # # FS: Function Signal
# # # TL: Ticker List

def Process(BS,TL,FS):
    # Lists of elements
    lists = ElementsList(BS,TL)
    
    # dataframe that contains the results of the processed files
    df_final = pd.DataFrame()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the threads for processing
        futures = [executor.submit(fetch_data_batch, list_batch, FS) for list_batch in lists[:BS]]

        # Iterate through the completed threads and concatenate the results
        for future in concurrent.futures.as_completed(futures):
            data = future.result()
            df_final = pd.concat([df_final, data])

    # Reset the index of the final DataFrame
    df_final = df_final.reset_index(drop=True)

    return df_final

In [11]:
historical_financial = Process(20,tkrs,1)

historical_financial

,ZAMP3.SA,N2ET34.SA,M1TA34.SA,AERI3.SA,ICBR3.SA,DOTZ3.SA,GOLL3.SA,ARML3.SA,MLAS3.SA,CBAV3.SA,...,TAEE4.SA,TAEE3.SA,SBSP3.SA,GEPA4.SA,GEPA3.SA,CESP6.SA,CESP5.SA,CESP3.SA,CMIG4.SA,CMIG3.SA
0,NaN,NaN,NaN,NaN,Quote not found for ticker symbol: ICBR3.SA,NaN,No fundamentals data found for any of the summ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
historical_balance = Process(20,tkrs,2)

historical_balance

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AllowanceForDoubtfulAccountsReceivable,AssetsHeldForSaleCurrent,AvailableForSaleSecurities,...,DefinedPensionBenefit,ForeignCurrencyTranslationAdjustments,InterestPayable,LiabilitiesHeldforSaleNonCurrent,PreferredSecuritiesOutsideStockEquity,PreferredSharesNumber,CurrentDeferredTaxesAssets,CurrentDeferredAssets,NonCurrentNoteReceivables,NotesReceivable
0,2019-12-31,12M,BRL,116629000.0,9.853200e+07,-48235000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,12M,BRL,205304000.0,2.201320e+08,-71737000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-31,12M,BRL,445286000.0,1.268770e+08,-103941000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-31,12M,BRL,336048000.0,0.000000e+00,-155715000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-31,12M,BRL,6729000.0,8.468000e+06,-13696000.0,NaN,-397000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,2022-12-31,12M,BRL,111557000.0,3.030059e+09,-93055000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,2019-12-31,12M,BRL,167774000.0,2.061882e+09,-38547000.0,NaN,NaN,NaN,NaN,...,43024000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1218,2020-12-31,12M,BRL,153346000.0,2.804373e+09,-54900000.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1219,2021-12-31,12M,BRL,83666000.0,2.344141e+09,-69655000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
historical_finandata = Process(20,tkrs,3)

historical_finandata

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [14]:
historical_data = Process(20,tkrs,4)

historical_data

,maxAge,regularMarketChangePercent,regularMarketChange,regularMarketTime,priceHint,regularMarketPrice,regularMarketDayHigh,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,...,underlyingSymbol,shortName,longName,currency,quoteSourceName,currencySymbol,fromCurrency,toCurrency,lastMarket,marketCap
0,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,...,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA,Quote not found for ticker symbol: BPAR3.SA
1,1,-0.001027,-0.059998,2023-07-12 15:22:39,2,58.36,59.0,58.05,8007,58.42,...,None,VISA INC DRN,Visa Inc.,BRL,Delayed Quote,R$,None,None,None,2447466561536
2,1,0.006794,0.559998,2023-07-12 14:15:41,2,82.99,83.29,82.99,17,82.43,...,None,MORGAN STAN DRN,Morgan Stanley,BRL,Delayed Quote,R$,None,None,None,693013774336
3,1,-0.003364,-0.209999,2023-07-12 15:22:39,2,62.22,62.33,61.9,470,62.43,...,None,MASTERCARD DRN,Mastercard Incorporated,BRL,Delayed Quote,R$,None,None,None,1864976039936
4,1,-0.001116,-0.080002,2023-07-12 15:26:33,2,71.59,72.0,70.87,49941,71.67,...,None,JPMORGAN DRN,JPMorgan Chase & Co.,BRL,Delayed Quote,R$,None,None,None,2126602305536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,...,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA,Quote not found for ticker symbol: CESP6.SA
375,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,...,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA,Quote not found for ticker symbol: CESP5.SA
376,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not found for ticker symbol: CESP3.SA,Quote not fo

Processing batches of diferent, first 1: historical financial

In [ ]:
# # Exporting Excel file Dataframes
# df.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/AtivosBr.xlsx',
#             sheet_name="stocks")

# dfbeta.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Beta.xlsx'
#             ,sheet_name="key_stats")

# addata.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer='/content/drive/MyDrive/Data/XLXS/RAW/Price.xlsx',
#             sheet_name="price") 

Local Machine Version

In [ ]:
# df.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/AtivosBR.xlsx',
#            sheet_name="stocks",
#            engine='openpyxl')

# dfbeta.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Beta.xlsx',
#                 sheet_name="key_stats")

# addata.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Financial.xlsx',
#             sheet_name="financial_data")

# dfprice.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Price.xlsx',
#             sheet_name="price")

# balance_sheet.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/Balance.xlsx',
#             sheet_name="balance")

In [ ]:
historical_data.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/tst/AtivosBR.xlsx',
           sheet_name="stocks",
           engine='openpyxl')

historical_finandata.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/tst/Beta.xlsx',
                sheet_name="key_stats")

historical_financial.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/tst/Financial.xlsx',
            sheet_name="financial_data")

historical_balance.to_excel(excel_writer=r'../notebooks/RAW_DATA/XLSX/tst/Price.xlsx',
            sheet_name="price")